In [1]:
# MAIN IMPORTS 
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter


# from torchviz import make_dot
import cv2
import numpy as np
import skimage
import imageio
import mpmath
import matplotlib.pyplot as plt
import seaborn as sns
from  PIL import Image
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import os
# https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html - documetnation on how to make a pytorch model

# Data preprocessing 
- dataloaders
- augmentation pipeline
## Add notes on this here (what is happening)

In [2]:
#data loader
# the images are loaded as float32 and normalised
# the mask is thresholded at 0.5 
""" the permute is needed since the format for image tensors must be (C, H, W)
But when we read from opencv the shape is (H, W, C)
and the mask must be of dim (1, H, W) since single channel - unsqueeze add this channel
"""
# returned as tensors 

class PuzzleDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(img_dir)
        self.masks = os.listdir(mask_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.masks[idx])

        # get the images
        image = cv2.imread(img_path)
        # cv2 gives BGR switch it to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        #get masks
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        #normalise the images and mask to float32
        image = image/255.0
        mask = mask / 255.0
        #threshold the mask
        
        mask = (mask > 0.5).astype(np.float32)
        mask = torch.tensor(mask)

        # for any data augmentation we want to do since training with small set and to avoid overfitting
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return torch.tensor(image, dtype=torch.float32).permute(2,0,1), mask.unsqueeze(0)

Loading the training dataset

In [3]:
train_dataset = PuzzleDataset(img_dir = "./images-1024x768/train/",
                            mask_dir = "./masks-1024x768/train/")
#since 10 images batches of 1 should be fine can do like batches of 2 i guess                        
train_loader = DataLoader(train_dataset,batch_size =1, shuffle=True)

# to visualise the images + masks 
image, mask = train_dataset[0]
image = (image.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
# Display the images
cv2.imshow('Image', image)


: 

In [ ]:
test_mask12 = cv2.imread("/images-1024x768/train/image-2.png")
plt.imshow("Image",test_mask12)

# 3.1 Unet Construction
## Add notes on this here (what is happening)

## Vairant 1 : Using `torch.nn.ConvTranspose2d` for upsampling

## Vairant 2: Using `torch.nn.Upsample` for bilinear upsampling

# 3.2 Training and evaluation 
### Training
- [ ] Implement the training loop with binary cross entropy loss for pixel-wise classification
- [ ] Adam Optimiser to update the model parameters
**Note** have regularly saved checkpoints and evaluate model in the validation set to avoid over fitting

### Weights and biases integration
Log the following:
- [ ] Track BCE loss over epochs
- [ ] IoU for both training and validation sets to monitor the segmentation performance

### Need to submit from the wandb dashboard
- Loss curves (training and validation over epochs)
- IoU curves (training and validation over epochs)

### Remember to comment on comment on whether the model is overfitting and how to recognise this and deal with the problem

### Evaluation
evaluate model on test set 
- accuracy
- precision
- recall
- F1 score
- IoU

# Select the best model based on the validation IoU and report its performance on the test set

# 4 Other architectures
